![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Vector Search with Redisvl
## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/01_redisvl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Prepare data

In this examples we will load a list of movie objects with the following attributes: `title`, `rating`, `description`, and `genre`.

For the vector part of our vector search we will embed the description so that user's can search for movies that best match what they're looking for.

**If you are running this notebook locally**, FYI you may not need to perform this step at all.

In [1]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/vector-search/resources .
!rm -rf temp_repo

Cloning into 'temp_repo'...
remote: Enumerating objects: 384, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 384 (delta 135), reused 151 (delta 74), pack-reused 137 (from 1)
Receiving objects: 100% (384/384), 64.50 MiB | 18.57 MiB/s, done.
Resolving deltas: 100% (159/159), done.


## Packages

In [2]:
# NBVAL_SKIP
%pip install -q redis redisvl numpy sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.0 MB/s eta 0:00:00


## Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [3]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [4]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client

In [5]:
from redis import Redis

client = Redis.from_url(REDIS_URL)

In [6]:
import json

with open("resources/movies.json", 'r') as file:
    movies = json.load(file)

In [7]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

# load model for embedding our movie descriptions
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def embed_text(model, text):
    return np.array(model.encode(text)).astype(np.float32).tobytes()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Note: convert embedding array to bytes for storage in Redis Hash data type
movie_data = [
    {
        **movie,
        "vector": embed_text(model, movie["description"])
    } for movie in movies
]

In [9]:
movie_data[0]

{'title': 'Explosive Pursuit',
 'genre': 'action',
 'rating': 7,
 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
 'vector': b'\x96f|=\x0fa\n;\xc4\x91\xb7;\x17\xcb~\xbd\x13e\xce\xbb\xd2\x16J=V\xa7?=\xc0v\x95<g\xfa\x06\xbe\x18Y\xcf=9\x07p=B\xdb\r\xbd\x9c\xf2H\xbdre\xc6<9\xdfa=\x868\x16\xbc\xcd\xd3\x13<5\xaa\x1c=\x13\xef\x89<\xb9\xb0-<\xa3\xb2\x9f\xbcU\x0b\xc3\xbd\x99NR=ol\xf7\xbcN>\x17\xbeo\x1c\x05\xb9$u\xbf<0\xe4b\xba\xd2\xa6\xa8\xbdt\xdc\xec\xbc_c%=\xd8\xe6r\xbb(OG=>(\x85=l@\xa2\xbc-Z\xd0\xbd2%K\xbd\xa0\xed\x94\xbco\xddH=\xed&F<\xc5*\xec<\x87\xd8\x8d\xbd\xceZ\x98<\x0c\xa3\xa3=Ig3\xbd\'\xcd\xbd\xbd\x0b%\xf7;\x04\xf5z=\x01\xb5\x8c=\x8a\x0e\xc6\xbdoI\x90\xbdU\x16\xbd;\x83\xe7\x0c\xbd\x053\xc9\xbct\xf8\xbb\xbc\xba&u\xbb/\x8f\xca<\x06\x80J=\x08\xaf*=\x8eOU\xbd\xcd\xf0\x95\xbc\x1e\x02\x19=\x17\xf4K<\xcd\xc2\t=F\x83\xac=\x97\xd7\xb8\xbd\xf6\xb5\x9c\xbd6\x85\x18=\x9ad&=\'3\xf8<\xf7\xf7\x88<)v\xf2\xbb/=[\xbd\xd4\xac\xee\xbb1:A\

## Define Redis index schema

In [10]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex

index_name = "movies"

schema = IndexSchema.from_dict({
  "index": {
    "name": index_name,
  },
  "fields": [
    {
        "name": "title",
        "type": "text",
    },
    {
        "name": "description",
        "type": "text",
    },
    {
        "name": "genre",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "rating",
        "type": "numeric",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "vector",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "hnsw",
            "datatype": "float32"
        }
    }
  ]
})


index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

In [11]:
index.info()

{'index_name': 'movies',
 'index_options': [],
 'index_definition': ['key_type',
  'HASH',
  'prefixes',
  ['rvl'],
  'default_score',
  '1'],
 'attributes': [['identifier',
   'title',
   'attribute',
   'title',
   'type',
   'TEXT',
   'WEIGHT',
   '1'],
  ['identifier',
   'description',
   'attribute',
   'description',
   'type',
   'TEXT',
   'WEIGHT',
   '1'],
  ['identifier',
   'genre',
   'attribute',
   'genre',
   'type',
   'TAG',
   'SEPARATOR',
   ',',
   'SORTABLE'],
  ['identifier',
   'rating',
   'attribute',
   'rating',
   'type',
   'NUMERIC',
   'SORTABLE',
   'UNF'],
  ['identifier',
   'vector',
   'attribute',
   'vector',
   'type',
   'VECTOR',
   'algorithm',
   'HNSW',
   'data_type',
   'FLOAT32',
   'dim',
   384,
   'distance_metric',
   'COSINE',
   'M',
   16,
   'ef_construction',
   200]],
 'num_docs': 0,
 'max_doc_id': 0,
 'num_terms': 0,
 'num_records': 0,
 'inverted_sz_mb': '0',
 'vector_index_sz_mb': '0.02032470703125',
 'total_inverted_index_b

## Populate index

In [12]:
index.load(movie_data)

['rvl:0b623c1614034752b4a5ca9b24996f2c',
 'rvl:96492c091c3b433ba906ef8e0332f598',
 'rvl:93f54dc9372f4e24a137fd98a8dbf4ae',
 'rvl:87c83d676188450c8ee0d9db614c4a0b',
 'rvl:6c7e6794915e40a183daf3dc809a8b79',
 'rvl:8b83d2019f5d4ba8aa86bbe1c3ed3822',
 'rvl:7eb6d663b9df4d48a6293ca31fdad8bb',
 'rvl:ce30db59503b481ca38a8590be32c61a',
 'rvl:4118ba0112fc4eac9bd88294a0d80ff6',
 'rvl:557472361db748d28333f52613ef79d3',
 'rvl:89b9cc2aea1942d0a26db09e569b033d',
 'rvl:bcf219313b424e53b41ed003fdff6c11',
 'rvl:eca73acfd9f9478eb1d8f97ba238ae4e',
 'rvl:5ffc9f405d5e4556a7ee22e3b5f5de6b',
 'rvl:6903f42d3dca4b6ca60c3303606911e8',
 'rvl:e08e720b63dc496895951de4ee0bb6bf',
 'rvl:3705fa202350432bb1780d62142aeb25',
 'rvl:eb29d0276be444c6afb017d6958a1385',
 'rvl:f4b2a387297d4688a4cce9456f55705d',
 'rvl:575607cbcb564fc19bde50717c40cf48']

## Index loaded now we can perform vector search

### basic vector search

In [13]:
from redisvl.query import VectorQuery

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre"],
    return_score=True
)

result = index.query(vec_query)
pd.DataFrame(result)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,id,vector_distance,title,rating,genre
0,rvl:93f54dc9372f4e24a137fd98a8dbf4ae,0.68577337265,Fast & Furious 9,6,action
1,rvl:eb29d0276be444c6afb017d6958a1385,0.801602959633,Despicable Me,7,comedy
2,rvl:e08e720b63dc496895951de4ee0bb6bf,0.812341928482,The Incredibles,8,comedy


### Hybrid filter vector search

Redis allows you to combine filter searches on fields within the index object allowing us to create more specific searches.

In [14]:
# Search for top 3 movies specifically in the action genre

from redisvl.query.filter import Tag

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre"],
    return_score=True
)

tag_filter = Tag("genre") == "action"

vec_query.set_filter(tag_filter)

result=index.query(vec_query)
pd.DataFrame(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,id,vector_distance,title,rating,genre
0,rvl:93f54dc9372f4e24a137fd98a8dbf4ae,0.68577337265,Fast & Furious 9,6,action
1,rvl:8b83d2019f5d4ba8aa86bbe1c3ed3822,0.820429563522,Mad Max: Fury Road,8,action
2,rvl:0b623c1614034752b4a5ca9b24996f2c,0.851705253124,Explosive Pursuit,7,action


In [15]:
# Search for top 3 movies specifically in the action genre with ratings at or above a 7

from redisvl.query.filter import Num

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

tag_filter = Tag("genre") == "action"
num_filter = Num("rating") >= 7
combined_filter = tag_filter & num_filter

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre"],
    return_score=True,
    filter_expression=combined_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,id,vector_distance,title,rating,genre
0,rvl:8b83d2019f5d4ba8aa86bbe1c3ed3822,0.820429563522,Mad Max: Fury Road,8,action
1,rvl:0b623c1614034752b4a5ca9b24996f2c,0.851705253124,Explosive Pursuit,7,action
2,rvl:557472361db748d28333f52613ef79d3,0.856359899044,The Avengers,8,action


In [16]:
# Search with full text search for movies that directly mention "criminal mastermind" in the description

from redisvl.query.filter import Text

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre", "description"],
    return_score=True
)

text_filter = Text("description") == "criminal mastermind"

vec_query.set_filter(text_filter)

result = index.query(vec_query)
pd.DataFrame(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,id,vector_distance,title,rating,genre,description
0,rvl:eb29d0276be444c6afb017d6958a1385,0.801602959633,Despicable Me,7,comedy,When a criminal mastermind uses a trio of orph...
1,rvl:7eb6d663b9df4d48a6293ca31fdad8bb,0.982345640659,The Dark Knight,9,action,"Batman faces off against the Joker, a criminal..."


In [17]:
# Vector search with wildcard match

from redisvl.query.filter import Text

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre", "description"],
    return_score=True
)

text_filter = Text("description") % "crim*"

vec_query.set_filter(text_filter)

result = index.query(vec_query)
pd.DataFrame(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,id,vector_distance,title,rating,genre,description
0,rvl:eb29d0276be444c6afb017d6958a1385,0.801602959633,Despicable Me,7,comedy,When a criminal mastermind uses a trio of orph...
1,rvl:e08e720b63dc496895951de4ee0bb6bf,0.812341928482,The Incredibles,8,comedy,"A family of undercover superheroes, while tryi..."
2,rvl:0b623c1614034752b4a5ca9b24996f2c,0.851705253124,Explosive Pursuit,7,action,A daring cop chases a notorious criminal acros...


In [18]:
# Vector search with fuzzy match filter

from redisvl.query.filter import Text

user_query = "Movies with central main character"

embedded_user_query = embed_text(model, user_query)

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre", "description"],
    return_score=True
)

# Note: fuzzy match is based on Levenshtein distance. Therefore, "hero" might return result for "her" as an example.
# See docs for more info https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/query_syntax/
text_filter = Text("description") % "%hero%"

vec_query.set_filter(text_filter)

result = index.query(vec_query)
pd.DataFrame(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,id,vector_distance,title,rating,genre,description
0,rvl:557472361db748d28333f52613ef79d3,0.737778544426,The Avengers,8,action,Earth's mightiest heroes come together to stop...
1,rvl:87c83d676188450c8ee0d9db614c4a0b,0.76883995533,Black Widow,7,action,Natasha Romanoff confronts her dark past and f...
2,rvl:575607cbcb564fc19bde50717c40cf48,0.897787928581,The Princess Diaries,6,comedy,Mia Thermopolis has just found out that she is...


## Range queries

Range queries allow you to set a pre defined "threshold" for which we want to return documents. This is helpful when you only want documents with a certain distance from the search query.

In [19]:
from redisvl.query import RangeQuery

user_query = "Family friendly fantasy movies"

embedded_user_query = embed_text(model, user_query)

range_query = RangeQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    return_fields=["title", "rating", "genre"],
    return_score=True,
    distance_threshold=0.8  # find all items with a semantic distance of less than 0.8
)

result = index.query(range_query)
pd.DataFrame(result)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,id,vector_distance,title,rating,genre
0,rvl:e08e720b63dc496895951de4ee0bb6bf,0.644702553749,The Incredibles,8,comedy
1,rvl:87c83d676188450c8ee0d9db614c4a0b,0.747987031937,Black Widow,7,action
2,rvl:eb29d0276be444c6afb017d6958a1385,0.750915408134,Despicable Me,7,comedy
3,rvl:6903f42d3dca4b6ca60c3303606911e8,0.75129878521,Shrek,8,comedy
4,rvl:3705fa202350432bb1780d62142aeb25,0.761669337749,"Monsters, Inc.",8,comedy
5,rvl:eca73acfd9f9478eb1d8f97ba238ae4e,0.778580129147,Aladdin,8,comedy


Like the queries above, we can also chain additional filters and conditional operators with range queries. The following adds an `and` condition that returns vector search within the defined range and with a rating at or above 8.

In [20]:
user_query = "Family friendly fantasy movies"

embedded_user_query = embed_text(model, user_query)

range_query = RangeQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    return_fields=["title", "rating", "genre"],
    distance_threshold=0.8  # find all items with a semantic distance of less than 0.7
)

numeric_filter = Num("rating") >= 8

range_query.set_filter(numeric_filter)

# in this case we want to do a simple filter search or the vector so we execute as a joint filter directly
result = index.query(range_query)
pd.DataFrame(result)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,id,vector_distance,title,rating,genre
0,rvl:e08e720b63dc496895951de4ee0bb6bf,0.644702553749,The Incredibles,8,comedy
1,rvl:6903f42d3dca4b6ca60c3303606911e8,0.75129878521,Shrek,8,comedy
2,rvl:3705fa202350432bb1780d62142aeb25,0.761669337749,"Monsters, Inc.",8,comedy
3,rvl:eca73acfd9f9478eb1d8f97ba238ae4e,0.778580129147,Aladdin,8,comedy


### Next steps

For more query examples with redisvl: [see here](https://github.com/redis/redis-vl-python/blob/main/docs/user_guide/hybrid_queries_02.ipynb)

In [21]:
# clean up!
# client.flushall()